In [5]:
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input, State, dash_table
from jupyter_dash import JupyterDash
import censusgeocode as cg
import plotly.figure_factory as ff
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from Partisian import get_state_part_score
from Life import get_age_life_fig
from sklearn.neighbors import KNeighborsClassifier
from plotly.subplots import make_subplots
from math import floor
import GraphGenerator
from LifeData import LifeData
import math
#from GraphGenerator import cultural_ontology

In [6]:
number = float(75.9)
rounded = round(math.floor(number/10))*10
print('Rounded Number :', rounded)

Rounded Number : 70


  Using cached dash-2.11.1-py3-none-any.whl (10.4 MB)



[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Attempting uninstall: dash
    Found existing installation: dash 2.9.3
    Uninstalling dash-2.9.3:
      Successfully uninstalled dash-2.9.3


In [3]:
def make_dummy_graph():
    fig = go.Figure(data=go.Scatter(
    x=[1, 2, 3, 4],
    y=[10, 11, 12, 13],
    mode='markers',
    marker=dict(size=[40, 60, 80, 100],
                color=[0, 1, 2, 3])))
    return fig

In [22]:

app = JupyterDash(prevent_initial_callbacks="initial_duplicate")

app.config.suppress_callback_exceptions = True

app.layout = html.Div(className="row-fluid", children=[
    html.H1(children='USA', style={'textAlign':'center'}),
    html.Div(dcc.Dropdown(['State','Cultural Ontology'], 'State', id = "divide_dropdown"), className = "span8 offset2"),
    html.Div(className="container-fluid", id="graph-container", children=[]),
    #html.Div(className="container-fluid", id="graph-container-2", children=[]),
    html.Button('Submit', id='make-graph', n_clicks=0)
])

@app.callback(Output('graph-container', 'children', allow_duplicate=True),
              Input('StateWide', 'clickData'),
              State('graph-container', 'children'),
              prevent_initial_call=True)
def StateClick(stateClick, div_children):
    if stateClick:
        state = stateClick["points"][0]["location"]
        ind = stateClick["points"][0]["hovertext"]
    else:
        state="NY"
        ind="Life Exp"
    life_data = LifeData()
    df = life_data.get_state_card_data(state)
    graphs = {"LifeBarChart":GraphGenerator.create_life_bar(ind, state)}
    div_children =  display_table(div_children, df, "state-card", new_row=False)
    return display_graph(div_children, graphs)



@app.callback(Output('graph-container', 'children'),
              Input('divide_dropdown', 'value'),
              State('graph-container', 'children'))
def change_main_view(divide_category, div_children):
    if divide_category == "State":
        graphs = {"StateWide":GraphGenerator.life_us_map("Life Exp")}
    else:
        graphs = {"Ontology":GraphGenerator.cultural_ontology()}
    #graphs = [make_dummy_graph()]
    return display_graph(div_children, graphs, size=6)


def display_table(parent, df, id, new_row=True, size=4):
    span_size = f"span{size}"
    existing_ids = {}
    for i in range(len(parent)):
        try:
            existing_ids[parent[i]['props']['children'][0]['props']['children']['props']['id']] = i
        except:
            None
    print(f"Existing ids: {existing_ids}")
    new_chart =html.Div(dash_table.DataTable(
                        id=id, page_size=10, 
                        data=df.to_dict('records'),
                        style_cell={'textAlign': 'left'},
                        
                        style_as_list_view=True),
                        className=span_size, id=str(id)+"_parent")
    if len(parent)>0 and id in existing_ids.keys(): 
        parent[existing_ids[id]]['props']['children'][0] = new_chart
    else:
        if new_row:
            row = html.Div(className="row", children=[])
            row.children.append(new_chart)
            parent.append(row)
        else:
            print(f"New Chart {new_chart}")
            parent[-1]['props']['children'].append(new_chart.to_plotly_json())
        
    
    return parent

def add_dash_element(parent, element, new_row=True, id=""):
    #Check if graph already exists
    
    
    None


def display_graph(parent, graphs, new_row=True, size=6):
    span_size = f"span{size}"
    
    new_row = html.Div(className="row", children=[])
    existing_ids = {}
    for i in range(len(parent)):
        try:
            existing_ids[parent[i]['props']['children'][0]['props']['children']['props']['id']] = i
        except:
            None
    
    for id, graph in graphs.items():
        new_chart = html.Div(dcc.Graph(figure=graph, id=id), className=span_size, id=str(id)+"_parent")
        if id in graphs.keys() and len(parent)>0 and id in existing_ids.keys(): 
            parent[existing_ids[id]]['props']['children'][0] = new_chart
        else:
            new_row.children.append(new_chart)
            parent.append(new_row)

    return parent

    


if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/


NY
    index State  Year         Indicator   Rate  Rank
0      31    NY  2020          Life Exp  77.70    35
3      31    NY  2020  Infant Mortality   4.08     6
11     31    NY  2021    Fertility Rate  54.10    12
20     31    NY  2021  Low Birth Weight   8.40    26
29     31    NY  2021         Premature   9.69    14
38     31    NY  2021        Teen Birth   9.10     9
47     31    NY  2021   Unmarried Birth  38.10    23
Existing ids: {'StateWide': 0}
New Chart Div(children=DataTable(data=[{'Year': 2020, 'Indicator': 'Life Exp', 'Rate': 77.7, 'Rank': 35}, {'Year': 2020, 'Indicator': 'Infant Mortality', 'Rate': 4.08, 'Rank': 6}, {'Year': 2021, 'Indicator': 'Fertility Rate', 'Rate': 54.1, 'Rank': 12}, {'Year': 2021, 'Indicator': 'Low Birth Weight', 'Rate': 8.4, 'Rank': 26}, {'Year': 2021, 'Indicator': 'Premature', 'Rate': 9.69, 'Rank': 14}, {'Year': 2021, 'Indicator': 'Teen Birth', 'Rate': 9.1, 'Rank': 9}, {'Year': 2021, 'Indicator': 'Unmarried Birth', 'Rate': 38.1, 'Rank': 23}], page_

In [5]:
a = {1:"test"}

In [6]:
if 1 in a.keys():
    print("weoo")

weoo


NY
    index State  Year         Indicator   Rate  Rank
0      31    NY  2020          Life Exp  77.70    35
3      31    NY  2020  Infant Mortality   4.08     6
11     31    NY  2021    Fertility Rate  54.10    12
20     31    NY  2021  Low Birth Weight   8.40    26
29     31    NY  2021         Premature   9.69    14
38     31    NY  2021        Teen Birth   9.10     9
47     31    NY  2021   Unmarried Birth  38.10    23
Existing ids: {'StateWide': 0}
New Chart Div(children=DataTable(data=[{'Year': 2020, 'Indicator': 'Life Exp', 'Rate': 77.7, 'Rank': 35}, {'Year': 2020, 'Indicator': 'Infant Mortality', 'Rate': 4.08, 'Rank': 6}, {'Year': 2021, 'Indicator': 'Fertility Rate', 'Rate': 54.1, 'Rank': 12}, {'Year': 2021, 'Indicator': 'Low Birth Weight', 'Rate': 8.4, 'Rank': 26}, {'Year': 2021, 'Indicator': 'Premature', 'Rate': 9.69, 'Rank': 14}, {'Year': 2021, 'Indicator': 'Teen Birth', 'Rate': 9.1, 'Rank': 9}, {'Year': 2021, 'Indicator': 'Unmarried Birth', 'Rate': 38.1, 'Rank': 23}], page_

In [ ]:

app = JupyterDash(prevent_initial_callbacks="initial_duplicate")

app.config.suppress_callback_exceptions = True

app.layout = html.Div(className="row-fluid", children=[
    html.H1(children='USA', style={'textAlign':'center'}),
    html.Div(dcc.Dropdown(['State','Cultural Ontology'], 'State', id = "divide_dropdown"), className = "span8 offset2"),
    html.Div(className="container-fluid", id="graph-container", children=[]),
    #html.Div(className="container-fluid", id="graph-container-2", children=[]),
    html.Button('Submit', id='make-graph', n_clicks=0)
])

@app.callback(Output('graph-container', 'children', allow_duplicate=True),
              Input('StateWide', 'clickData'),
              State('graph-container', 'children'),
              prevent_initial_call=True)
def StateClick(stateClick, div_children):
    if stateClick:
        state = stateClick["points"][0]["location"]
        ind = stateClick["points"][0]["hovertext"]
    else:
        state="NY"
        ind="Life Exp"
    life_data = LifeData()
    df = life_data.get_state_card_data(state)
    graphs = {"LifeBarChart":GraphGenerator.create_life_bar(ind, state), "StateCard":life_data.get_state_card_data(state)}
    
    #div_children =  table_factory(div_children, df, "state-card", new_row=True)
    return graph_factory(div_children, graphs)



@app.callback(Output('graph-container', 'children'),
              Input('divide_dropdown', 'value'),
              State('graph-container', 'children'))
def change_main_view(divide_category, div_children):
    if divide_category == "State":
        graphs = {"StateWide":GraphGenerator.life_us_map("Life Exp")}
    else:
        graphs = {"Ontology":GraphGenerator.cultural_ontology()}
    #graphs = [make_dummy_graph()]
    return graph_factory(div_children, graphs)


def table_factory(parent, df, id, new_row=True):
    existing_ids = {}
    for i in range(len(parent)):
        try:
            existing_ids[parent[i]['props']['children'][0]['props']['children']['props']['id']] = i
        except:
            None
    print(f"Existing ids: {existing_ids}")
    new_chart =html.Div(dash_table.DataTable(id=id, page_size=10, data=df.to_dict('records')), id=str(id)+"_parent")
    if len(parent)>0 and id in existing_ids.keys(): 
        parent[existing_ids[id]]['props']['children'][0] = new_chart
    else:
        new_row = html.Div(className="row", children=[])
        new_row.children.append(new_chart)
        parent.append(new_row)
    return parent


def graph_factory(parent, graphs, new_row=True, size=12):
    span_size = f"span_{size}"
    new_row = html.Div(className="row", children=[])
    existing_ids = {}
    for i in range(len(parent)):
        try:
            existing_ids[parent[i]['props']['children'][0]['props']['children']['props']['id']] = i
        except:
            None
    
    for id, graph in graphs.items():
        new_chart = html.Div(dcc.Graph(figure=graph, id=id), className=span_size, id=str(id)+"_parent")
        if id in graphs.keys() and len(parent)>0 and id in existing_ids.keys(): 
            parent[existing_ids[id]]['props']['children'][0] = new_chart
        else:
            
            new_row.children.append(new_chart)
            parent.append(new_row)

    return parent

    


if __name__ == '__main__':
    app.run_server(debug=True)
